In [2]:
import pandas as pd
import yfinance as yf
import pendulum
import csv

# Define the start and end dates for the loop
import datetime

In [13]:
data = yf.download(
  tickers=['AAPL'],
  # start=start_time,
  # end=end_time,
  period='max',
  interval='1d'
)

# Drop second level of column names
data.columns = data.columns.droplevel(1)

csv_file = "stock_data_alternative.csv"
with open(csv_file, mode="w", newline="") as file:
		writer = csv.writer(file)
		# Write the header
		writer.writerow(["Date"] + list(data.columns))
		# Write the data
		for index, row in data.iterrows():
				writer.writerow([index] + list(row))


[*********************100%***********************]  1 of 1 completed


In [15]:
# Get the current date
end_date = datetime.date.today()

# Calculate the date 30 days before the current date
days_before = datetime.timedelta(days=30)
start_date = current_date - days_before

# Create an empty DataFrame to store the data
# all_data = pd.read_csv('stock_data/BTC_2010-2011.csv', index_col=0, header=[0, 1]).sort_index(axis=1)
all_data = pd.DataFrame()

# Loop through the range of dates
for date in pd.date_range(start_date, end_date, freq='D'):
	# Define the start and end times for the data retrieval
	start_time = date.replace(hour=0, minute=0, second=0)
	end_time = date.replace(hour=23, minute=59, second=59)

	# Retrieve the data for the specified date range
	data = yf.download(
		tickers=['BTC-USD'],
		start=start_time,
		end=end_time,
		# period='1d',
		interval='1m'
	)

	# Flatten the column headers to remove the ticker
	if (isinstance(data.columns, pd.MultiIndex)):
			data.columns = [col[0] for col in data.columns]  # Retain only the first level

	# Append the data to the DataFrame
	all_data = pd.concat([all_data, data])

# data = yf.download(
# 		tickers=['BTC-USD'],
# 		# start=start_time,
# 		# end=end_time,
# 		period='1d',
# 		interval='1m'
# 	)

# data = data.drop(data.index[0])
# print(data)

csv_file = "stock_data_alternative.csv"
with open(csv_file, mode="w", newline="") as file:
		writer = csv.writer(file)
		# Write the header
		writer.writerow(["Date"] + list(all_data.columns))
		# Write the data
		for index, row in all_data.iterrows():
				writer.writerow([index] + list(row))



[*********************100%***********************]  1 of 1 completed

1 Failed download:
['BTC-USD']: YFPricesMissingError('$%ticker%: possibly delisted; no price data found  (1m 2024-10-21 00:00:00 -> 2024-10-21 23:59:59) (Yahoo error = "1m data not available for startTime=1729468800 and endTime=1729555199. The requested range must be within the last 30 days.")')


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

In [104]:
data = pd.read_csv('stock_data/BTC_2010-2011.csv', index_col=0, header=[0, 1]).sort_index(axis=1)
data.shape

(34104, 6)

In [106]:
def clean_data(df):
    print("Initial data information:")
    print(df.info())

    # Remove unnecessary columns (if there are columns named 'Unnamed')
    df = df.loc[:, ~df.columns.get_level_values(1).str.contains('^Unnamed')]

    # Handle the 'timestamp' column (if present)
    if ('timestamp', '') in df.columns:
        # Convert to datetime format
        df[('timestamp', '')] = pd.to_datetime(df[('timestamp', '')], errors='coerce')  
        
        # Remove rows with missing timestamp values
        df = df.dropna(subset=[('timestamp', '')]) 

    # Fill missing values using forward fill and backward fill methods
    df = df.fillna(method='ffill').fillna(method='bfill')

    # Normalize 'close' values between 0 and 1 if present
    if ('close', '') in df.columns:
        df[('close', '')] = (df[('close', '')] - df[('close', '')].min()) / (df[('close', '')].max() - df[('close', '')].min())

    # Print out the data after cleaning
    print("Data after cleaning:")
    print(df.head())

    return df

data

Date,Adj Close,Close,High,Low,Open,Volume
2024-10-21 00:00:00+00:00,69009.3671875,69009.3671875,69009.3671875,69009.3671875,69009.3671875,0.0
2024-10-21 00:01:00+00:00,69002.984375,69002.984375,69002.984375,69002.984375,69002.984375,1761280.0
2024-10-21 00:02:00+00:00,68993.679688,68993.679688,68993.679688,68993.679688,68993.679688,0.0
2024-10-21 00:03:00+00:00,68999.046875,68999.046875,68999.046875,68999.046875,68999.046875,3409920.0
2024-10-21 00:04:00+00:00,68973.656250,68973.656250,68973.656250,68973.656250,68973.656250,12740608.0
2024-10-21 00:06:00+00:00,68966.562500,68966.562500,68966.562500,68966.562500,68966.562500,3717120.0
...,...,...,...,...,...,...
2024-11-19 17:16:00+00:00,92840.359375,92840.359375,92840.359375,92840.359375,92840.359375,0.0
2024-11-19 17:17:00+00:00,92706.343750,92706.343750,92706.343750,92706.343750,92706.343750,0.0
2024-11-19 17:18:00+00:00,92672.953125,92672.953125,92672.953125,92672.953125,92672.953125,28139520.0


In [107]:
clean_data(data)

Initial data information:
<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 34104 entries, 2024-10-21 00:01:00+00:00 to 2024-11-19 17:20:00+00:00
Data columns (total 6 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   (Adj Close, 69009.3671875)  34104 non-null  float64
 1   (Close, 69009.3671875)      34104 non-null  float64
 2   (High, 69009.3671875)       34104 non-null  float64
 3   (Low, 69009.3671875)        34104 non-null  float64
 4   (Open, 69009.3671875)       34104 non-null  float64
 5   (Volume, 0.0)               34104 non-null  float64
dtypes: float64(6)
memory usage: 1.8 MB
None
Data after cleaning:
Date                          Adj Close         Close          High  \
2024-10-21 00:00:00+00:00 69009.3671875 69009.3671875 69009.3671875   
2024-10-21 00:01:00+00:00  69002.984375  69002.984375  69002.984375   
2024-10-21 00:02:00+00:00  68993.679688  68993.679688  68993.679688   
2024-10-21 00

Date,Adj Close,Close,High,Low,Open,Volume
2024-10-21 00:00:00+00:00,69009.3671875,69009.3671875,69009.3671875,69009.3671875,69009.3671875,0.0
2024-10-21 00:01:00+00:00,69002.984375,69002.984375,69002.984375,69002.984375,69002.984375,1761280.0
2024-10-21 00:02:00+00:00,68993.679688,68993.679688,68993.679688,68993.679688,68993.679688,0.0
2024-10-21 00:03:00+00:00,68999.046875,68999.046875,68999.046875,68999.046875,68999.046875,3409920.0
2024-10-21 00:04:00+00:00,68973.656250,68973.656250,68973.656250,68973.656250,68973.656250,12740608.0
2024-10-21 00:06:00+00:00,68966.562500,68966.562500,68966.562500,68966.562500,68966.562500,3717120.0
...,...,...,...,...,...,...
2024-11-19 17:16:00+00:00,92840.359375,92840.359375,92840.359375,92840.359375,92840.359375,0.0
2024-11-19 17:17:00+00:00,92706.343750,92706.343750,92706.343750,92706.343750,92706.343750,0.0
2024-11-19 17:18:00+00:00,92672.953125,92672.953125,92672.953125,92672.953125,92672.953125,28139520.0
